<a href="https://colab.research.google.com/github/SCS-Technology-and-Innovation/DDDM/blob/main/decisionscience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 53.3 MB/s eta 0:00:00


In [8]:
from pulp import *
import pandas as pd
import numpy as np
n_warehouses = 2
n_customers = 4

# Cost Matrix
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])
# Demand Matrix
cust_demands = np.array([35000, 22000, 18000, 30000])

# Supply Matrix
warehouse_supply = np.array([65000, 80000])
model = LpProblem("Supply-Demand-Problem", LpMinimize)
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1, n_warehouses+1)]
variable_names.sort()
print("Variable Indices:", variable_names)
DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound= 0 )
allocation = np.array(DV_variables).reshape(2,4)
print("Decision Variable/Allocation Matrix: ")
print(allocation)
obj_func = lpSum(allocation*cost_matrix)
print(obj_func)
model +=  obj_func
print(model)
#Supply Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)
print("Total Cost:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error could not find value")

Variable Indices: ['11', '12', '13', '14', '21', '22', '23', '24']
Decision Variable/Allocation Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]
X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24
Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

X_11 + X_12 + X_13 + X_14 <= 65000
X_21 + X_22 + X_23 + X_24 <= 80000
X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000
Optimal
Total Cost: 195000.0
X_11 = 35000.0
X_12 = 22000.0
X_13 = 8000.0
X_14 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 10000.0
X_24 = 30000.0


[Tutorial](https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0#:~:text=Linear%20programming%20or%20linear%20optimization,varying%20set%20of%20decision%20variables.)